In [ ]:
import glob
import os
import sys
import numpy as np
import cv2
import random


In [ ]:
# # import yolo
# from IPython import display
# display.clear_output()

In [ ]:
# import tensorflow
import time

In [ ]:
import ultralytics
from ultralytics import YOLO
# yolo=YOLO("yolov8n.pt")


In [ ]:
from PIL import Image

In [ ]:
try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass
import carla

In [ ]:
# def process(image):
#     i=np.array(image.raw_data)
#     i2=i.reshape((600,600,4))
#     i3=i2[:,:,:3]
#    # cv2.imshow("",i3)
#     # cv2.waitKey(1)
#     normi=(i3/255.0)
#     opi=cv2.cvtColor((normi*255).astype(np.uint8),cv2.COLOR_RGB2BGR)
#     return opi

In [ ]:
actor_list=[]

In [ ]:
#import random
# client=carla.Client('localhost',2000)
# client.set_timeout(2000.0)
# world=client.get_world()
# bpl=world.get_blueprint_library()
# bp=bpl.filter('model3')[0]
# print(bp)
# spp=random.choice(world.get_map().get_spawn_points())
# # sp=spp[0]
# vl=world.spawn_actor(bp,spp)
# actor_list.append(vl)
# vp=vl.get_transform()

client=carla.Client('localhost',2000)
client.set_timeout(2000.0)
world=client.get_world()
bpl=world.get_blueprint_library()
bp=bpl.filter('model3')[0]
print(bp)
spp=world.get_map().get_spawn_points()
sp=spp[10]
vl=world.spawn_actor(bp,sp)
actor_list.append(vl)

In [ ]:
#control the vehicle:
def control_vehicle(traffic_light_state):
    control = carla.VehicleControl()
    if traffic_light_state == "red":
        control.throttle = 0.0
        control.brake = 1.0
    elif traffic_light_state == "green":
        control.throttle = 1.0
        control.brake = 0.0
    else:
        # Default behavior if the traffic light state is not known
        control.throttle = 0.5
        control.brake = 0.0
    vl.apply_control(control)

In [ ]:
yolo = YOLO('yolov8n.pt')

# Secondary model (yolotraficlight) for traffic light detection
yolotraficlight = YOLO("C:/Users/ASUS/Downloads/CARLA_0.9.8/WindowsNoEditor/PythonAPI/examples/best1.pt")

import supervision as sv

rf = Roboflow(api_key="PqaW7bzyyhdeZntUMVOJ")
project = rf.workspace().project("object-recognition-dddrb")
model = project.version(2).model

def detect_traffic_light_state(image):
    # Perform traffic light detection using yolotraficlight
    result = yolotraficlight(image)[0]
    print("bla bla bla bla bla bla")
    # Determine traffic light state (green, red, yellow)
    for box in result.boxes:
        state_label = result.names[int(box.cls[0])] 
        print("Detected traffic light state:", state_label)
        return state_label
    return "Unknown"

# Callback function for processing each frame
def cm_callback(image):
    # Convert image to numpy array
    img = np.array(image.raw_data).reshape((600, 600, 4))
    img_color = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)  # Convert to BGR format
    
    # Perform object detection using YOLOv8 on color image
    result = yolo(img_color)[0]

    # Iterate over detected objects
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        
        # Check if the detected object is a traffic light
        label = result.names[int(box.cls[0])]
        if label == "traffic light":
            # Crop the region of interest containing the traffic light from the color image
            traffic_light_roi = img_color

            
            # Detect traffic light state using yolotraficlight
            traffic_light_state = detect_traffic_light_state(traffic_light_roi)
            traffic_light_state = detect_traffic_light_state(traffic_light_roi)
            control_vehicle(traffic_light_state)
            
            # Draw bounding box and label on the original image
            cv2.rectangle(img_color, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img_color, label + ": " + traffic_light_state, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 2)
        else:
            # Draw bounding box and label on the original image for non-traffic light objects
            cv2.rectangle(img_color, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img_color, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 2)
    
    # Save or display the annotated image
    frame_id = str(time.time())
    file_path = os.path.join('data_carla', f'dete_obj_{frame_id}.png')
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    cv2.imwrite(file_path, img_color)
    #cv2.imshow("Annotated Image", img_color)
    #cv2.waitKey(1)  # Use 1 ms waitKey to update the display window

In [ ]:
#vl.set_autopilot(True)

In [ ]:
#vehicle controle whitout autopilote
#vl.apply_control(carla.VehicleControl(throttle=1.0,steer=0.0))

In [ ]:
bp2=bpl.find('sensor.camera.rgb')
bp2.set_attribute("image_size_x",f'{600}')
bp2.set_attribute("image_size_y",f'{600}')
bp2.set_attribute('fov','110')
#position of the sensor
ssp=carla.Transform(carla.Location(x=2.5,z=0.7))
sn=world.spawn_actor(bp2,ssp,attach_to=vl)
actor_list.append(sn)

In [ ]:
#vehicle controle without autopilote
vl.apply_control(carla.VehicleControl(throttle=1.0,steer=0.0))
sn.listen(cm_callback)

In [ ]:
#vl.destroy()
#sn.destroy()
for actor in actor_list:
     actor.destroy()